# BigEarthNet - Linear Probing

In [45]:
import torch
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import fbeta_score
from sklearn.metrics import hamming_loss
from convert_labels import convert_labels
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

In [32]:
train_feat = torch.load('./features/feat-train-bigearthnet.pth')
train_labels = torch.load('./features/labels-train-bigearthnet.pth')
test_feat = torch.load('./features/feat-test-bigearthnet.pth')
test_labels = torch.load('./features/labels-test-bigearthnet.pth')

flattened_train_features = np.array(train_feat.view(-1, 768))
flattened_train_labels = np.array(train_labels.view(-1, 43))
flattened_train_labels = convert_labels(flattened_train_labels)
flattened_test_features = np.array(test_feat.view(-1, 768))
flattened_test_labels = np.array(test_labels.view(-1, 43))
flattened_test_labels = convert_labels(flattened_test_labels)

In [33]:
print(flattened_train_features.shape)
print(flattened_train_labels.shape)

(269696, 768)
(269696, 19)


In [34]:
train_count = flattened_train_features.shape[0]
test_count = flattened_test_features.shape[0]

## Random Forest

In [35]:
rf_classifier = RandomForestClassifier(
                        n_estimators=20, 
                        criterion='gini', 
                        random_state=42, 
                        max_depth=None,
                        min_samples_split=10,
                        min_samples_leaf=10,
                        max_features='sqrt',
                        max_leaf_nodes=20000,
                        bootstrap=False,
                        n_jobs=-1)
rf_classifier.fit(flattened_train_features[:train_count], flattened_train_labels[:train_count])

RandomForestClassifier(bootstrap=False, max_leaf_nodes=20000,
                       min_samples_leaf=10, min_samples_split=10,
                       n_estimators=20, n_jobs=-1, random_state=42)

In [36]:
test_preds = rf_classifier.predict(flattened_test_features[:test_count])

In [37]:
flattened_test_labels[:test_count].shape

(125888, 19)

In [41]:
print("Random Forest Result")
print("f2_macro: ", fbeta_score(test_preds, flattened_test_labels[:test_count], average='macro', beta=2))
print("f2_micro: ", fbeta_score(test_preds, flattened_test_labels[:test_count], average='micro', beta=2))
print("hamming loss: ", hamming_loss(test_preds, flattened_test_labels[:test_count]))

Random Forest Result
f2_macro:  0.30808670099136604
f2_micro:  0.5739335299262083
hamming loss:  0.12245011438739196


## Linear Probing

In [89]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("device: ", device)

device:  mps


In [97]:
torch.manual_seed(0)
scaler = StandardScaler()
X_train = scaler.fit_transform(flattened_train_features)
X_test = scaler.transform(flattened_test_features)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(flattened_train_labels, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(flattened_test_labels, dtype=torch.float32).to(device)

In [98]:
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)

In [99]:
class LinearLayer(nn.Module):
    def __init__(self):
        super(LinearLayer, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(768, 19),
        )

    def forward(self, x):
        return self.layers(x)

model = LinearLayer().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [100]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for data, targets in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 0.3106
Epoch 2, Loss: 0.3037
Epoch 3, Loss: 0.2779
Epoch 4, Loss: 0.3183
Epoch 5, Loss: 0.2558
Epoch 6, Loss: 0.2903
Epoch 7, Loss: 0.2807
Epoch 8, Loss: 0.2766
Epoch 9, Loss: 0.2934
Epoch 10, Loss: 0.3057
Epoch 11, Loss: 0.2800
Epoch 12, Loss: 0.2738
Epoch 13, Loss: 0.2972
Epoch 14, Loss: 0.2681
Epoch 15, Loss: 0.3114
Epoch 16, Loss: 0.3271
Epoch 17, Loss: 0.2897
Epoch 18, Loss: 0.2994
Epoch 19, Loss: 0.2713
Epoch 20, Loss: 0.2951


In [102]:
model.eval()
preds = model(X_test) > 0
preds = preds.cpu().detach().numpy()

In [103]:
print("Linear Probing result")
print("f2_macro: ", fbeta_score(preds, flattened_test_labels, average='macro', beta=2))
print("f2_micro: ", fbeta_score(preds, flattened_test_labels, average='micro', beta=2))
print("hamming loss: ", hamming_loss(preds, flattened_test_labels))

Linear Probing result
f2_macro:  0.3261981658380211
f2_micro:  0.5623474174697499
hamming loss:  0.12445523840740641


## Multi Layer Perceptron

In [105]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(768, 384),
            nn.ReLU(),
            nn.Linear(384, 192),
            nn.ReLU(),
            nn.Linear(192, 100),
            nn.ReLU(),
            nn.Linear(100, 19)
        )

    def forward(self, x):
        return self.layers(x)

model = MLP().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [106]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for data, targets in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
    
    # Print loss every epoch
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 0.2533
Epoch 2, Loss: 0.2930
Epoch 3, Loss: 0.2420
Epoch 4, Loss: 0.2327
Epoch 5, Loss: 0.2442
Epoch 6, Loss: 0.2666
Epoch 7, Loss: 0.2732
Epoch 8, Loss: 0.2699
Epoch 9, Loss: 0.2202
Epoch 10, Loss: 0.2281
Epoch 11, Loss: 0.2324
Epoch 12, Loss: 0.2303
Epoch 13, Loss: 0.2325
Epoch 14, Loss: 0.2302
Epoch 15, Loss: 0.2754
Epoch 16, Loss: 0.2369
Epoch 17, Loss: 0.2518
Epoch 18, Loss: 0.2930
Epoch 19, Loss: 0.2139
Epoch 20, Loss: 0.2409


In [107]:
model.eval()
preds = model(X_test) > 0
preds = preds.cpu().detach().numpy()

print("MLP result")
print("f2_macro: ", fbeta_score(preds, flattened_test_labels, average='macro', beta=2))
print("f2_micro: ", fbeta_score(preds, flattened_test_labels, average='micro', beta=2))
print("hamming loss: ", hamming_loss(preds, flattened_test_labels))

MLP result
f2_macro:  0.4479707769040759
f2_micro:  0.6311090558256047
hamming loss:  0.1124809354346721
